# 多模态数据验证与可视化

验证数据格式是否正确，并可视化前 8 条数据

In [1]:
import pandas as pd
import os
import json
from pathlib import Path
from IPython.display import display, Image as IPImage, HTML
import re

## 1. 加载数据

In [2]:
# 数据路径
data_dir = Path("/home/vgc7798/projects_p32509/userdata/zheyu/verl_Uky")
train_file = data_dir / "train.parquet"

print(f"数据路径: {train_file}")
print(f"文件大小: {train_file.stat().st_size / 1024:.2f} KB")
print()

数据路径: /home/vgc7798/projects_p32509/userdata/zheyu/verl_Uky/train.parquet
文件大小: 38.59 KB



In [3]:
# 加载 parquet 数据
df = pd.read_parquet(train_file)

print(f"✓ 成功加载数据")
print(f"样本总数: {len(df)}")
print(f"列名: {list(df.columns)}")
print()

✓ 成功加载数据
样本总数: 1216
列名: ['data_source', 'prompt', 'images', 'ability', 'reward_model', 'extra_info']



## 2. 数据统计信息

In [4]:
# 统计每个样本的图片数量
image_counts = df['images'].apply(len)

print("=" * 60)
print("数据集统计")
print("=" * 60)
print(f"\n总样本数: {len(df)}")
print(f"\n每个样本的图片数量:")
print(f"  最小: {image_counts.min()}")
print(f"  最大: {image_counts.max()}")
print(f"  平均: {image_counts.mean():.2f}")
print(f"  中位数: {image_counts.median():.0f}")

# 统计答案分布
answers = df['reward_model'].apply(lambda x: x.get('ground_truth') if isinstance(x, dict) else None)
print(f"\n答案分布:")
for ans, count in answers.value_counts().items():
    print(f"  {ans}: {count} ({count/len(df)*100:.1f}%)")

# 统计 <image> 标签数量
def count_image_tags(row):
    content = row['prompt'][0]['content']
    return content.count('<image>')

image_tag_counts = df.apply(count_image_tags, axis=1)
print(f"\n<image> 标签数量:")
print(f"  最小: {image_tag_counts.min()}")
print(f"  最大: {image_tag_counts.max()}")
print(f"  平均: {image_tag_counts.mean():.2f}")

print()

数据集统计

总样本数: 1216

每个样本的图片数量:
  最小: 2
  最大: 5
  平均: 3.50
  中位数: 4

答案分布:
  A: 1216 (100.0%)

<image> 标签数量:
  最小: 2
  最大: 5
  平均: 3.50



## 3. 显示前 8 条数据的详细信息

In [5]:
def display_sample(idx, sample):
    """显示单个样本的详细信息"""
    print("=" * 80)
    print(f"样本 {idx + 1}")
    print("=" * 80)
    
    # 基本信息
    extra_info = sample['extra_info']
    print(f"\n【ID】: {extra_info['id']}")
    print(f"【类别】: {extra_info.get('category', 'N/A')}")
    print(f"【格式】: {extra_info.get('format', 'N/A')}")
    
    # Prompt 内容
    prompt = sample['prompt'][0]
    print(f"\n【Prompt 角色】: {prompt['role']}")
    print(f"【Prompt 内容】:")
    print("-" * 80)
    content = prompt['content']
    # 高亮显示 <image> 标签
    highlighted = content.replace('<image>', '🖼️ <image> 🖼️')
    print(highlighted)
    print("-" * 80)
    
    # 图片信息
    images = sample['images']
    print(f"\n【图片列表】 (共 {len(images)} 张):")
    for i, img_path in enumerate(images):
        exists = os.path.exists(img_path)
        status = "✓" if exists else "❌"
        print(f"  {status} [{i+1}] {img_path}")
    
    # 奖励模型信息
    reward_model = sample['reward_model']
    print(f"\n【奖励模型】:")
    print(f"  正确答案: {reward_model.get('ground_truth')}")
    print(f"  评分方式: {reward_model.get('style')}")
    print(f"  选项数量: {reward_model.get('num_choices')}")
    
    # 验证 <image> 标签数量和图片数量是否匹配
    image_tag_count = content.count('<image>')
    image_count = len(images)
    match_status = "✓" if image_tag_count == image_count else "❌"
    print(f"\n【验证】:")
    print(f"  {match_status} <image> 标签数: {image_tag_count}")
    print(f"  {match_status} 图片路径数: {image_count}")
    if image_tag_count == image_count:
        print(f"  ✓ 数量匹配！")
    else:
        print(f"  ❌ 数量不匹配！")
    
    print("\n")

In [6]:
# 显示前 8 条数据
num_samples = min(8, len(df))
print(f"\n显示前 {num_samples} 条数据:\n")

for idx in range(num_samples):
    sample = df.iloc[idx]
    display_sample(idx, sample)


显示前 8 条数据:

样本 1

【ID】: id_0
【类别】: ['human', 'egocentric', 'view change', 'forward', 'living room']
【格式】: MCQ

【Prompt 角色】: user
【Prompt 内容】:
--------------------------------------------------------------------------------
Based on the image, which view represents the resulting state? 🖼️ <image> 🖼️

Choices:
A. 🖼️ <image> 🖼️
B. 🖼️ <image> 🖼️
C. 🖼️ <image> 🖼️
D. 🖼️ <image> 🖼️

Please select the correct answer.
--------------------------------------------------------------------------------

【图片列表】 (共 5 张):
  ✓ [1] /home/vgc7798/zheyu_b1222/example.jpeg
  ✓ [2] /home/vgc7798/zheyu_b1222/example.jpeg
  ✓ [3] /home/vgc7798/zheyu_b1222/example.jpeg
  ✓ [4] /home/vgc7798/zheyu_b1222/example.jpeg
  ✓ [5] /home/vgc7798/zheyu_b1222/example.jpeg

【奖励模型】:
  正确答案: A
  评分方式: rule
  选项数量: 4

【验证】:
  ✓ <image> 标签数: 5
  ✓ 图片路径数: 5
  ✓ 数量匹配！


样本 2

【ID】: id_1
【类别】: ['human', 'egocentric', 'view change', 'forward', 'living room']
【格式】: MCQ

【Prompt 角色】: user
【Prompt 内容】:
------------------------------

## 4. 模拟 verl 的数据处理流程

In [7]:
def simulate_verl_processing(sample):
    """模拟 verl 如何处理数据"""
    print("=" * 80)
    print("模拟 verl 数据处理流程")
    print("=" * 80)
    
    # 步骤 1: 读取 prompt
    messages = sample['prompt']
    print("\n【步骤 1】读取 messages:")
    print(f"  类型: {type(messages)}")
    print(f"  长度: {len(messages)}")
    
    # 步骤 2: 处理 <image> 标签
    print("\n【步骤 2】处理 <image> 标签:")
    content = messages[0]['content']
    print(f"  原始内容 (前100字符): {content[:100]}...")
    
    # verl 会将 <image> 分割并转换为多模态格式
    segments = re.split("(<image>)", content)
    segments = [item for item in segments if item != ""]
    
    content_list = []
    for segment in segments:
        if segment == "<image>":
            content_list.append({"type": "image"})
        else:
            content_list.append({"type": "text", "text": segment})
    
    print(f"\n  转换后的 content_list (共 {len(content_list)} 个元素):")
    for i, item in enumerate(content_list[:10]):  # 只显示前10个
        if item['type'] == 'image':
            print(f"    [{i}] 🖼️  IMAGE")
        else:
            text_preview = item['text'][:40].replace('\n', '↵')
            print(f"    [{i}] 📝  TEXT: '{text_preview}...'")
    
    # 步骤 3: 加载图片
    print("\n【步骤 3】加载图片:")
    images = sample['images']
    print(f"  图片数量: {len(images)}")
    print(f"  图片路径:")
    for i, img_path in enumerate(images):
        exists = "存在" if os.path.exists(img_path) else "不存在"
        print(f"    [{i}] {img_path} ({exists})")
    
    # 步骤 4: 获取 ground truth
    print("\n【步骤 4】获取 ground truth:")
    reward_model = sample['reward_model']
    ground_truth = reward_model.get('ground_truth')
    print(f"  正确答案: {ground_truth}")
    
    print("\n" + "=" * 80)
    print("✓ verl 数据处理流程模拟完成")
    print("=" * 80)
    
    return content_list, images, ground_truth

In [8]:
# 对第一个样本进行处理模拟
print("\n对第一个样本进行 verl 处理流程模拟:\n")
sample = df.iloc[0]
content_list, images, ground_truth = simulate_verl_processing(sample)


对第一个样本进行 verl 处理流程模拟:

模拟 verl 数据处理流程

【步骤 1】读取 messages:
  类型: <class 'numpy.ndarray'>
  长度: 1

【步骤 2】处理 <image> 标签:
  原始内容 (前100字符): Based on the image, which view represents the resulting state? <image>

Choices:
A. <image>
B. <imag...

  转换后的 content_list (共 11 个元素):
    [0] 📝  TEXT: 'Based on the image, which view represent...'
    [1] 🖼️  IMAGE
    [2] 📝  TEXT: '↵↵Choices:↵A. ...'
    [3] 🖼️  IMAGE
    [4] 📝  TEXT: '↵B. ...'
    [5] 🖼️  IMAGE
    [6] 📝  TEXT: '↵C. ...'
    [7] 🖼️  IMAGE
    [8] 📝  TEXT: '↵D. ...'
    [9] 🖼️  IMAGE

【步骤 3】加载图片:
  图片数量: 5
  图片路径:
    [0] /home/vgc7798/zheyu_b1222/example.jpeg (存在)
    [1] /home/vgc7798/zheyu_b1222/example.jpeg (存在)
    [2] /home/vgc7798/zheyu_b1222/example.jpeg (存在)
    [3] /home/vgc7798/zheyu_b1222/example.jpeg (存在)
    [4] /home/vgc7798/zheyu_b1222/example.jpeg (存在)

【步骤 4】获取 ground truth:
  正确答案: A

✓ verl 数据处理流程模拟完成


## 5. 验证数据格式完整性

In [9]:
print("=" * 80)
print("数据格式完整性检查")
print("=" * 80)

issues = []

# 检查必需列
required_columns = ['prompt', 'images', 'reward_model', 'ability', 'data_source']
missing_cols = [col for col in required_columns if col not in df.columns]
if missing_cols:
    issues.append(f"缺少必需列: {missing_cols}")
else:
    print("\n✓ 所有必需列都存在")

# 检查 <image> 标签和图片数量是否匹配
mismatch_count = 0
for idx, row in df.iterrows():
    content = row['prompt'][0]['content']
    image_tag_count = content.count('<image>')
    image_count = len(row['images'])
    if image_tag_count != image_count:
        mismatch_count += 1

if mismatch_count > 0:
    issues.append(f"{mismatch_count} 个样本的 <image> 标签数量和图片数量不匹配")
else:
    print("✓ 所有样本的 <image> 标签数量和图片数量都匹配")

# 检查图片文件是否存在
all_image_paths = set()
for images in df['images']:
    all_image_paths.update(images)

missing_images = []
for img_path in all_image_paths:
    if not os.path.exists(img_path):
        missing_images.append(img_path)

if missing_images:
    issues.append(f"{len(missing_images)} 个图片文件不存在")
else:
    print(f"✓ 所有 {len(all_image_paths)} 个唯一图片文件都存在")

# 最终结论
print("\n" + "=" * 80)
if not issues:
    print("\n🎉 所有检查都通过！数据格式正确，可以开始训练。\n")
else:
    print("\n⚠️  发现以下问题:\n")
    for i, issue in enumerate(issues, 1):
        print(f"  {i}. {issue}")
    print()
print("=" * 80)

数据格式完整性检查

✓ 所有必需列都存在
✓ 所有样本的 <image> 标签数量和图片数量都匹配
✓ 所有 1 个唯一图片文件都存在


🎉 所有检查都通过！数据格式正确，可以开始训练。



## 6. 生成数据格式总结报告

In [10]:
print("\n" + "=" * 80)
print("数据格式总结报告")
print("=" * 80)

print(f"\n📊 数据集信息:")
print(f"  - 总样本数: {len(df)}")
print(f"  - 数据源: {df.iloc[0]['data_source']}")
print(f"  - 能力标签: {df.iloc[0]['ability']}")

print(f"\n🖼️  图片信息:")
print(f"  - 每样本图片数: {image_counts.min()} ~ {image_counts.max()} (平均 {image_counts.mean():.2f})")
print(f"  - 唯一图片数: {len(all_image_paths)}")
print(f"  - 所有图片存在: {'是' if not missing_images else '否'}")

print(f"\n📝 Prompt 格式:")
print(f"  - 包含 <image> 占位符: 是")
print(f"  - <image> 标签数匹配图片数: {'是' if mismatch_count == 0 else '否'}")
print(f"  - 平均 prompt 长度: {df['prompt'].apply(lambda x: len(x[0]['content'])).mean():.0f} 字符")

print(f"\n🎯 标注信息:")
print(f"  - 包含 ground truth: 是")
print(f"  - 答案分布: {dict(answers.value_counts())}")
print(f"  - 奖励模型类型: {df.iloc[0]['reward_model']['style']}")

print(f"\n✅ 格式兼容性:")
print(f"  - verl 格式要求: 符合")
print(f"  - Qwen2.5-VL 格式: 兼容")
print(f"  - 多模态处理: 支持")

print("\n" + "=" * 80)
print("\n🚀 数据已准备就绪，可以开始训练！")
print("\n运行命令:")
print("  cd /home/vgc7798/projects_p32509/userdata/zheyu/verl_Uky")
print("  bash examples/grpo_trainer/run_qwen2_5_vl-3b_fake_dataset.sh vllm")
print("\n" + "=" * 80)


数据格式总结报告

📊 数据集信息:
  - 总样本数: 1216
  - 数据源: fake_vlm_dataset
  - 能力标签: spatial_reasoning

🖼️  图片信息:
  - 每样本图片数: 2 ~ 5 (平均 3.50)
  - 唯一图片数: 1
  - 所有图片存在: 是

📝 Prompt 格式:
  - 包含 <image> 占位符: 是
  - <image> 标签数匹配图片数: 是
  - 平均 prompt 长度: 185 字符

🎯 标注信息:
  - 包含 ground truth: 是
  - 答案分布: {'A': np.int64(1216)}
  - 奖励模型类型: rule

✅ 格式兼容性:
  - verl 格式要求: 符合
  - Qwen2.5-VL 格式: 兼容
  - 多模态处理: 支持


🚀 数据已准备就绪，可以开始训练！

运行命令:
  cd /home/vgc7798/projects_p32509/userdata/zheyu/verl_Uky
  bash examples/grpo_trainer/run_qwen2_5_vl-3b_fake_dataset.sh vllm

